In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pwd

### 1.Importar Librerías necesarias

In [ ]:
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from glob import glob
from tqdm.notebook import tqdm
from ast import literal_eval
import os
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 22})
from sklearn.metrics import accuracy_score
from skimage import exposure
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import cv2
from matplotlib.patches import Rectangle
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, DenseNet121
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import models
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import tensorflow.keras.backend as K
from tensorflow.math import confusion_matrix
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import os
import glob
import shutil
import sys
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
from IPython.display import Image
from keras.applications import Xception
%matplotlib inline

In [ ]:
pwd

In [ ]:
#import os
#import glob

#files = glob.glob('/kaggle/working/*')
#for f in files:
    #os.remove(f)

### 2. Cargar los datos (Ficheros csv, y carpetas ficheros de imágenes) Exploración y preparación de datos

In [ ]:
df_image = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
display(df_image.head(3))
print(df_image.shape)

In [ ]:
df_study = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
display(df_study.head(3))
print(df_study.shape)

In [ ]:
df_sampleSub = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
display(df_sampleSub.head(3))
print(df_sampleSub.shape)

In [ ]:
df_study['id'] = df_study['id'].str.replace('_study',"")
df_study.rename({'id': 'StudyInstanceUID'},axis=1, inplace=True)
df_study.head(3)

In [ ]:
df_train = df_image.merge(df_study, on='StudyInstanceUID')
df_train.head(3)

In [ ]:
df_train.loc[df_train['Negative for Pneumonia']==1, 'study_label'] = 'negative'
df_train.loc[df_train['Typical Appearance']==1, 'study_label'] = 'typical'
df_train.loc[df_train['Indeterminate Appearance']==1, 'study_label'] = 'indeterminate'
df_train.loc[df_train['Atypical Appearance']==1, 'study_label'] = 'atypical'
df_train.drop(['Negative for Pneumonia','Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance'], axis=1, inplace=True)
df_train['id'] = df_train['id'].str.replace('_image', '.jpg')
df_train['image_label'] = df_train['label'].str.split().apply(lambda x : x[0])
df_train.head(3)

In [ ]:
print(df_train.shape)

In [ ]:
df_size = pd.read_csv('/kaggle/input/covid-jpg-512/size.csv')
df_size.head(3)

In [ ]:
df_train = df_train.merge(df_size, on='id')
df_train.head(3)

### Vamos a realizar el mismo análisis pero con tres variables de salida. Tomado como referencia la variable "study_label" clasificamos como negativo, tipico (neumonía) y el resto creamos clase otros

In [ ]:
def map_values(row, values_dict):
    return values_dict[row]

values_dict = {'typical': 'Yes', 'negative': 'None', 'indeterminate': 'otros', 'atypical': 'otros'}

df_train['label_opacity'] = df_train['study_label'].apply(map_values, args = (values_dict,))



In [ ]:
df_train.head(3)

In [ ]:
print(len(df_train))

In [ ]:
df_train.groupby(['label_opacity']).size().reset_index(name='Nuevas clases')

In [ ]:
import os, os.path

DIR = '../input/covid-jpg-512/train'
print (len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

ruta_test="/kaggle/input/covid-jpg-512/test"
ruta_train="/kaggle/input/covid-jpg-512/train"
test_img=[]
train_img=[]
img_size=224

for img in os.listdir(ruta_train):
  img = cv2.imread(os.path.join(ruta_train,img))
  img_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  img_gray_resize=cv2.resize(img_gray,(img_size,img_size))
  train_img.append([img_gray_resize])

print(len(train_img))

In [ ]:
plt.figure()
plt.imshow(np.squeeze(train_img[10]))
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
for img in os.listdir(ruta_test):
  img = cv2.imread(os.path.join(ruta_test,img))
  img_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  img_gray_resize=cv2.resize(img_gray,(img_size,img_size))
  test_img.append([img_gray_resize])

print(len(test_img))

In [ ]:
plt.figure()
plt.imshow(np.squeeze(test_img[10]))
plt.colorbar()
plt.grid(False)
plt.show()

### 3. Visualición imágenes train con cajas que identifican las anomalías.

In [ ]:
n = 20
train_dir = '/kaggle/input/covid-jpg-512/train'
fig, axs = plt.subplots(4, 5, figsize=(20,20))
fig.subplots_adjust(hspace=.2, wspace=.2)
axs = axs.ravel()
for i in range(n):
    img = cv2.imread(os.path.join(train_dir, df_train['id'][i]))
    axs[i].imshow(img)
    if type(df_train['boxes'][i])==str:
        boxes = literal_eval(df_train['boxes'][i])
        for box in boxes:
            axs[i].add_patch(Rectangle((box['x']*(512/df_train['dim1'][i]), box['y']*(512/df_train['dim0'][i])), box['width']*(512/df_train['dim1'][i]), box['height']*(512/df_train['dim0'][i]), fill=0, color='y', linewidth=2))
            axs[i].set_title(df_train['study_label'][i])
    else:
        axs[i].set_title(df_train['study_label'][i])

### Tratamiento de las imágenes y partición de datos en Train y Validación

In [ ]:
def preprocess_image(img):
  equ_img = exposure.equalize_hist(img)
  return equ_img

im= cv2.imread('/kaggle/input/covid-jpg-512/train/000a312787f2.jpg')
im2 = preprocess_image(im)
res = np.concatenate((im/255, im2), axis=1)
plt.imshow(res)
plt.show()

In [ ]:
img_size = 224
batch_size = 16

image_generator = ImageDataGenerator(
        validation_split=0.2,
        #rotation_range=20,
        horizontal_flip = True,
        zoom_range = 0.1,
        #shear_range = 0.1,
        brightness_range = [0.8, 1.1],
        fill_mode='nearest',
        preprocessing_function=preprocess_image
)

image_generator_valid = ImageDataGenerator(validation_split=0.2,preprocessing_function=preprocess_image)

train_generator = image_generator.flow_from_dataframe(
        dataframe = df_train,
        directory='../input/covid-jpg-512/train',
        x_col = 'id',
        y_col =  'label_opacity',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        subset='training', seed = 10) 

valid_generator=image_generator_valid.flow_from_dataframe(
    dataframe = df_train,
    directory='../input/covid-jpg-512/train',
    x_col = 'id',
    y_col = 'label_opacity',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='validation', shuffle=False,  seed=10) 



In [ ]:
base_model = Xception(weights=None,input_shape=(img_size,img_size,3), include_top=False)
base_model.load_weights('../input/keras-pretrained-models/xception_weights_tf_dim_ordering_tf_kernels_notop.h5')
base_out = base_model.output
avg = keras.layers.GlobalAveragePooling2D()(base_out)
bath=keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(avg)
output = keras.layers.Dense(3, activation="softmax")(bath) 
model = keras.Model(inputs=base_model.input, outputs=output)

In [ ]:
for layer in base_model.layers: 
    layer.trainable = False
    
optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit_generator(train_generator, validation_data=valid_generator, epochs=10)

In [ ]:
# Congelamos los peso del modelo base para sacar el máximo partido del modelo preentrenado
for layer in base_model.layers: 
    layer.trainable = True

#Importo EarlyStopping
from keras.callbacks import EarlyStopping
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    
optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit_generator(train_generator, validation_data=valid_generator, epochs=30,callbacks=[callback],verbose=1)

In [ ]:
actual =  valid_generator.labels
preds = np.argmax(model.predict(valid_generator), axis=1)
cfmx = confusion_matrix(actual, preds)
acc = accuracy_score(actual, preds)
print ('Test Accuracy:', acc )
print('Confusion matrix:', cfmx)

In [ ]:
os.makedirs("./models", exist_ok=True)
model.save('./models/model_3labels_xception_2.h5')

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("../input/modelo-creado-3-labels/model_3labels_xception.h5")

In [ ]:
preds = np.argmax(model.predict(valid_generator), axis=1)

In [ ]:
preds.shape